In [2]:
import random
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate
import math
# import tensorflow as tf
# from tf.transformations import euler_from_quaterniont

In [3]:
# Determine the x range, z range, and height (in meters) of the Azure Kinect
# the Azure Kinect's height when mounted on the robot will be the fixed y value

# Generate random linear functions

def rand_linear():
    # for horizontal, m = 0
    # m = random.uniform(-10, 10)
    MAX_SLOPE = 5 #adjust later
    m = random.uniform(-2, 2)
    b = random.uniform(-MAX_SLOPE, MAX_SLOPE)
    return m,b

# fewer extra points/limit on how close they are to the line
def extra_pts():
    MAX_SCATTERED = 20
    scattered_x = []
    scattered_y = []
    num_points = random.randrange(0, MAX_SCATTERED)
    for i in range(num_points):
        x = random.uniform(-10.0, 10.0)
        y = random.uniform(-10.0, 10.0)
        scattered_x.append(x)
        scattered_y.append(y)
    scattered_z = [0] * len(scattered_x)

    return scattered_x, scattered_y, scattered_z

    
def spacedpts(x, y, Nmarks):
    data_ratio = plt.gca().get_data_ratio()
    dydx = np.gradient(y, x[1])
    dxdx = np.gradient(x, x[1]) * data_ratio
    arclength = scipy.integrate.cumtrapz(math.sqrt(dydx**2 + dxdx**2), x, initial=0)
    marks = np.linspace(0, max(arclength), Nmarks)
    markx = np.interp(marks, arclength, x)
    marky = np.interp(markx, x, y)
    return markx, marky

def rand_data():
    MAX_PEOPLE = 21 #fix later
    num_points = random.randrange(3, MAX_PEOPLE)
    MIN_SPACING = 3 #fix later
    MAX_SPACING = 4 #fix later
    spacing = random.uniform(MIN_SPACING, MAX_SPACING)
    MIN_START = -5 #minimum x range for kinect
    start_x = random.uniform(MIN_START, 5)
    m,b = rand_linear()

    # for every element in the array    
    
    ### for equidistant points on a curve/line
    # all_x = np.linspace(start_x, 100, 1000)
    # all_y = []
    # for i in range(len(all_x)):
    #     all_y.append(m*all_x[i] + b)
    # x,y = spaced_pts(all_x, all_y, num_points)
    ####
    
    x = []
    for i in range(num_points):
        # add a random variation to the spacing
        actual_spacing = spacing + random.uniform(-0.25, 0.25)
        x_offset = math.sqrt(actual_spacing**2 / (m**2 + 1))
        x.append(start_x + i*x_offset)
    
    # for every element in the array
    y = []
    scattered_x, scattered_y, scattered_z = extra_pts()

    for i in range(len(x)):
        y_offset = random.uniform(-0.25, 0.25)
        actual_y = (m*x[i] + b) + y_offset
        y.append(actual_y)

    # add random extra points
    # for i in range(len(scattered_x)):
    #     x.append(scattered_x[i])
    # for i in range(len(scattered_y)):
    #     y.append(scattered_y[i])

    z = [0] * len(y)
    
    return x,y,z

In [4]:
def get_bearing(lat1, long1, lat2, long2):
    dLon = (long2 - long1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    brng = np.arctan2(x,y)
    return brng

model_input = []
model_expected_output = []

for i in range(1000):
    #fig = plt.figure(figsize=(12, 12))
    # fig2 = plt.figure(figsize=(4, 4))
    # #ax = fig.add_subplot(projection='3d')
    # ax2 = fig2.add_subplot()

    # ax.set_xlim(-20,20)
    # ax.set_ylim(-20,20)
    # ax.set_zlim(-1,1)

    # ax.set_xlabel('x')
    # ax.set_ylabel('y')
    # ax.set_zlabel('z')

    # ax2.set_xlim(-50,50)
    # ax2.set_ylim(-50,50)
    # ax2.set_xlabel('x')
    # ax2.set_ylabel('y')

    x, y, z = rand_data()

    theta = []
    
    flip = random.randint(0, 1)
    for i in range(1,len(x)):
        rand_angle = math.radians(random.uniform(-15,15))
        if (flip == 0):
            actual_theta = get_bearing(x[i], y[i], x[i-1], y[i-1])   
        else:
            actual_theta = get_bearing(x[i-1], y[i-1], x[i], y[i])
        theta.append(actual_theta + rand_angle)

    theta.append(theta[0] if flip == 0 else theta[-1])

    u = [math.cos(t) for t in theta] # pointing in +x direction
    v = [math.sin(t) for t in theta] # pointing in +y direction

    #ax.quiver(x, y, z, u, v, 0, length=2, normalize=True, arrow_length_ratio=0.2)
    # ax2.quiver(x, y, u, v)

    trial = []

    if flip == 1:
        join_label = [x[0], y[0], u[0], v[0]]
        for i in range(1,len(x)):
            temp = [x[i], y[i], u[i], v[i]]
            trial.extend(temp)
    else:
        join_label = [x[-1], y[-1], u[-1], v[-1]]
        for i in range(len(x)-1):
            temp = [x[i], y[i], u[i], v[i]]
            trial.extend(temp)

    trial.extend([0] * (80 - len(trial)))
    model_input.extend(np.array(trial).reshape(1, 80))
    model_expected_output.extend(np.array(join_label).reshape(1,4))
    # print (flip)
    # print (join_label)

# print (model_input)
np.savetxt('model_input.csv', model_input, delimiter=",")  
np.savetxt('model_expected_output.csv', model_expected_output, delimiter=",")  
